In [21]:
library("gdalcubes")
library("rstac")
library("tibble")
library("sf")
library("geojsonsf")
library("stringr")

In [22]:
s = stac("https://planetarycomputer.microsoft.com/api/stac/v1/")

In [23]:
b <- c(-2009488, -715776, 1401061, 2597757)
bounds <- matrix(c(b[1],b[2],
                   b[1],b[4],
                   b[3],b[4],
                   b[3],b[2],
                   b[1],b[2]
                ),ncol=2, byrow=TRUE)

resolution <- 1000
epsg <- 32198
tt<-st_sfc(st_polygon(list(bounds)))
st_crs(tt) = epsg
bounds_ll <- tt |>
  st_transform(crs = 4326) |>
  sfc_geojson() |>
  jsonlite::fromJSON()
bounds_ll

$type
[1] "Polygon"

$coordinates
, , 1

          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] -90.01264 -108.9475 -38.70515 -53.26646 -90.01264

, , 2

         [,1]     [,2]     [,3]     [,4]     [,5]
[1,] 34.99047 62.15906 64.71252 36.33666 34.99047

In [25]:
it_obj <- s |>
    stac_search(intersects=bounds_ll, collections=c('cop-dem-glo-90'),limit=5000) |> post_request()
#it_obj |> items_sign(sign_fn = sign_planetary_computer())
st<-stac_image_collection(it_obj$features,asset_names=c("data"))
st
#
#

Warning message in stac_image_collection(it_obj$features, asset_names = c("data")):
“STAC asset with name 'data' does not include eo:bands metadata and will be considered as a single band source”


Image collection object, referencing 1077 images with 1  bands
Images:
                                      name      left      top   bottom
1 Copernicus_DSM_COG_30_N64_00_W054_00_DEM -54.00083 65.00042 64.00042
2 Copernicus_DSM_COG_30_N64_00_W053_00_DEM -53.00083 65.00042 64.00042
3 Copernicus_DSM_COG_30_N64_00_W052_00_DEM -52.00083 65.00042 64.00042
4 Copernicus_DSM_COG_30_N64_00_W051_00_DEM -51.00083 65.00042 64.00042
5 Copernicus_DSM_COG_30_N64_00_W050_00_DEM -50.00083 65.00042 64.00042
6 Copernicus_DSM_COG_30_N64_00_W049_00_DEM -49.00083 65.00042 64.00042
      right            datetime       srs
1 -53.00083 2021-04-22T00:00:00 EPSG:4326
2 -52.00083 2021-04-22T00:00:00 EPSG:4326
3 -51.00083 2021-04-22T00:00:00 EPSG:4326
4 -50.00083 2021-04-22T00:00:00 EPSG:4326
5 -49.00083 2021-04-22T00:00:00 EPSG:4326
6 -48.00083 2021-04-22T00:00:00 EPSG:4326
[ omitted 1071 images ] 

Bands:
  name offset scale unit nodata image_count
1 data      0     1                    1077


In [26]:
v = cube_view(srs = paste0("EPSG:",epsg),  extent = list(t0 = "2021-01-01", t1 = "2021-12-31",
              left = b[1], right = b[3],  top = b[4], bottom = b[2]),
              dx = resolution, dy = resolution, dt = "P1Y",aggregation = "max", resampling = "near")

gdalcubes_options(threads = 8)
mosaic = raster_cube(st, v, chunking = c(1, 5000, 5000)) |>
    select_bands(c("data"))
    #reduce_time(c("max(data)"))


In [27]:
mosaic |> 
    write_tif(
        dir = "/home/jovyan/",
        prefix = "cop-dem-glo-1km",
        COG = TRUE,
        creation_options = list('compress'='deflate')
    )
    #plot(zlim=c(-10,1000))